In [4]:
import pandas as pd
import math

# --- 설정 ---
# 읽을 CSV 파일 이름
csv_file_name = "../data_csv/11_11/춘천시업종_좌표포함.csv"

# 생성할 SQL 파일 이름
sql_file_name = "insert_chuncheon_businesses_with_coords.sql"

# 생성할 테이블 이름 (따옴표 없음)
table_name = "chuncheon_businesses_with_coords"
# ----------------

def clean_value_mysql(value):
    """
    Python 데이터를 SQL 문자열로 안전하게 변환합니다.
    - None 또는 NaN -> 'NULL'
    - 문자열 -> 작은따옴표로 감싸고, 내부의 작은따옴표 이스케이프
    """
    if pd.isna(value):
        return "NULL"
    elif isinstance(value, str):
        # MySQL은 '를 ''로, \를 \\로 이스케이프합니다.
        cleaned_value = value.replace("\\", "\\\\").replace("'", "''")
        return f"'{cleaned_value}'"
    else:
        # 숫자(float, int 등)는 문자열로 변환
        return str(value)

# 1. CSV 파일 읽기 (한글 인코딩 처리 및 첫 번째 열 인덱스로 지정)
try:
    # index_col=0 추가: CSV의 첫 번째 열(Unnamed: 0)을 인덱스로 사용
    df = pd.read_csv(csv_file_name, encoding='cp949', index_col=0)
except UnicodeDecodeError:
    df = pd.read_csv(csv_file_name, encoding='utf-8-sig', index_col=0)
except Exception as e:
    print(f"파일 읽기 오류: {e}")
    df = pd.DataFrame() # 오류 시 빈 데이터프레임

if not df.empty:
    print(f"'{csv_file_name}' 파일 읽기 성공. {len(df)}개 행 발견.")
    print(f"읽어들인 컬럼: {list(df.columns)}")
    
    # 2. CREATE TABLE 쿼리 생성
    # 컬럼명 리스트 (따옴표 없음)
    columns = [col for col in df.columns]
    
    # '춘천시업종_좌표포함.csv'에 맞게 예상 컬럼
    expected_columns = ['업종', '업소명', '주소', '연락처', '위도', '경도']
    if list(columns) != expected_columns:
        print(f"경고: CSV 컬럼명({columns})이 예상({expected_columns})과 다릅니다.")
        print("CREATE TABLE 문과 INSERT 로직이 실패할 수 있습니다.")

    # 컬럼명에서 따옴표(백틱) 제거
    column_sql = ", ".join(columns)
    
    create_table_sql = f"""
-- 춘천시 업종 (좌표 포함) 테이블 생성 (MySQL용, 따옴표 없음)
DROP TABLE IF EXISTS {table_name};
CREATE TABLE {table_name} (
    업종 VARCHAR(255) CHARACTER SET utf8mb4,
    업소명 VARCHAR(255) CHARACTER SET utf8mb4,
    주소 VARCHAR(255) CHARACTER SET utf8mb4,
    연락처 VARCHAR(50),
    위도 DOUBLE,
    경도 DOUBLE
);
\n"""

    # 3. SQL 파일 쓰기 시작
    try:
        with open(sql_file_name, 'w', encoding='utf-8') as f:
            # 파일 상단에 데이터베이스 선택 주석 추가
            f.write("-- 사용할 데이터베이스를 선택하세요 (예: USE your_database_name;)\n")
            f.write("SET NAMES utf8mb4;\n\n") # 인코딩 설정 추가
            
            # CREATE TABLE 문 쓰기
            f.write(create_table_sql)
            f.write("\n")
            
            values_batch = []
            
            # INSERT 문 시작 부분 (테이블명과 컬럼명에 따옴표 없음)
            insert_prefix = f"INSERT INTO {table_name} ({column_sql}) VALUES \n"
            
            # 4. DataFrame을 행별로 순회하며 VALUES 생성
            for row in df.itertuples(index=False):
                try:
                    cleaned_row = [
                        clean_value_mysql(getattr(row, '업종')),
                        clean_value_mysql(getattr(row, '업소명')),
                        clean_value_mysql(getattr(row, '주소')),
                        clean_value_mysql(getattr(row, '연락처')),
                        clean_value_mysql(getattr(row, '위도')),  # '위도' 컬럼 추가
                        clean_value_mysql(getattr(row, '경도'))   # '경도' 컬럼 추가
                    ]
                    values_batch.append(f"({', '.join(cleaned_row)})")
                except AttributeError as ae:
                    print(f"오류: DataFrame 행에서 예기치 않은 속성 접근 실패 - {ae}")
                    print(f"CSV 파일의 컬럼명이 {expected_columns}와(과) 일치하는지 확인하세요.")
                    break # 오류 발생 시 중단
            
            # 5. INSERT 쿼리 파일에 쓰기
            if values_batch:
                # 모든 데이터를 하나의 INSERT 문으로 합쳐서 씀
                f.write(insert_prefix + ",\n".join(values_batch) + ";\n\n")
                
        print(f"'{sql_file_name}' 파일 생성이 완료되었습니다.")
        print(f"총 {len(values_batch)}개의 행이 파일에 저장되었습니다.")

    except Exception as e:
        print(f"SQL 파일 생성 중 오류: {e}")


'../data_csv/11_11/춘천시업종_좌표포함.csv' 파일 읽기 성공. 34개 행 발견.
읽어들인 컬럼: ['업종', '업소명', '주소', '연락처', '위도', '경도']
'insert_chuncheon_businesses_with_coords.sql' 파일 생성이 완료되었습니다.
총 34개의 행이 파일에 저장되었습니다.
